In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec

client = MongoClient()
client = MongoClient('da1.eecs.utk.edu', 27017) 
#Connects to the MongoDB, make sure youre SSH'ed into the docker

i =0
relevancedata = []

for post in client['twitter']['LabeledStatuses_MiscRelevant_C'].find({}):  
    #Extracts all the entries with a sentiment field
    #Updates each data with the name of the collection it comes from
    relevancedata.append(post) 

df = pd.DataFrame(relevancedata)
# pulls data into the Pandas DataFrame


print(df.head(3))
print("Setting vairables")


                        _id                  id  \
0  5c505763d2512c412edea9fe  958545105337704449   
1  5c505763d2512c412edea9ff  913729938737549312   
2  5c505763d2512c412edeaa00  920915671831142401   

                                            original  \
0  {'Tweet': 'When the people of Iran rose up aga...   
1  {'Tweet': 'Manchester Cityâs Danilo: âIâ...   
2  {'Tweet': 'Michael Jordan scores again, this t...   

                               original_file  original_line tags  \
0  LABELED_DATA/Manually_labelled_data_2.csv              2  [0]   
1  LABELED_DATA/Manually_labelled_data_2.csv              3  [0]   
2  LABELED_DATA/Manually_labelled_data_2.csv              4  [0]   

                                                text  
0  When the people of Iran rose up against the cr...  
1  Manchester Citys Danilo: Im looking forward, I...  
2  Michael Jordan scores again, this time with hi...  
Setting vairables


In [2]:
print(df.columns.values)

['_id' 'id' 'original' 'original_file' 'original_line' 'tags' 'text']


In [3]:
def remove_list(funnyboi):
    return funnyboi[0]

df.tags = df.tags.apply(remove_list)

print(df.tags.value_counts())



1    6185
0    4697
Name: tags, dtype: int64


In [4]:
print(df.isna().sum())

_id              0
id               0
original         0
original_file    0
original_line    0
tags             0
text             0
dtype: int64


In [5]:
df.text = df.text.astype(str)

In [6]:
# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(df, "text", ["tags"])

print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
x.score(verbose=True)


Scoring Model
tags



Accuracy Score:  0.8155963302752294
Label Score:  [0.7821256500253873, 0.839824835369295]


[[0.7821256500253873, 0.839824835369295], 0.8238532110091743]

In [7]:
tweets = pd.read_csv("tweets.csv")
print(tweets.head(3))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.
            Source   categories                       _id created_at  \
0  Statuses_Irma_C          NaN  5c409973ec4ee50898175e81  12/3/2017   
1  Statuses_Irma_A  ['utility']  5c11a746ec4ee522100a5b3b  9/15/2017   
2  Statuses_Irma_A      ['gov']  5c0c5cf2ec4ee5221008263d   9/8/2017   

   emotion_ml  opinion_ml                                               text  \
0         0.0         1.0  We will add a few more warmer-than-average day...   
1         0.0         1.0  I am happy to speak with someone at @dukeenerg...   
2         0.0         1.0  Consider it done! Great work! #9PMRoutine http...   

   Relevance  1st Code  2nd Code  
0        0.0       NaN       NaN  
1        1.0       9.0      27.0  
2        0.0       NaN       NaN  


In [8]:
print(tweets.Relevance.value_counts())

1.0    377
0.0    194
Name: Relevance, dtype: int64


In [9]:
print(tweets.isna().sum())

Source           7
categories    1197
_id              7
created_at       1
emotion_ml      10
opinion_ml      10
text            16
Relevance     1185
1st Code      1701
2nd Code      1698
dtype: int64


In [10]:
tweets.Relevance.fillna(0, inplace=True)

print(tweets.Relevance.value_counts())

0.0    1379
1.0     377
Name: Relevance, dtype: int64


In [11]:
tweets  =  tweets[["text", "Relevance"]]
print(tweets.isna().sum())

text         16
Relevance     0
dtype: int64


In [12]:
tweets = tweets.dropna()
print(tweets.isna().sum())

text         0
Relevance    0
dtype: int64


In [13]:
# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(tweets, "text", ["Relevance"])

print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
x.score(verbose=True)


Scoring Model
Relevance



Accuracy Score:  0.8080459770114944
Label Score:  [0.8549189518036255, 0.6963007904601084]


[[0.8549189518036255, 0.6963007904601084], 0.8850574712643678]

In [14]:
frames = []
df  =  df[["text", "tags"]]

frames.append(df)
frames.append(tweets)
result = pd.concat(frames)
print(result.columns.values)

['Relevance' 'tags' 'text']


In [15]:
result['tags'].fillna(result['Relevance'], inplace=True)
result  =  result[["text", "tags"]]
print(result.columns.values)

['text' 'tags']


In [16]:
print(result.isna().sum())

text    0
tags    0
dtype: int64


In [17]:
print(result.tags.value_counts())

1.0    6562
0.0    6060
Name: tags, dtype: int64


In [18]:
result['tags'].fillna(0.0, inplace=True)
print(result.isna().sum())

text    0
tags    0
dtype: int64


In [19]:
# We pass the class 3 fields:
# 1. The DataFrame
# 2. The X value, the Text (Pandas Series)
# 3. The Y values, the labels that correspond to the text (Pandas Series)
#                   It can be a list of the names of the columns or one as a string

x = doc2vec(result, "text", ["tags"])

print("Scoring Model")

#returns scores of each labels accuracy from the first column passed into 
# the Y or 3 arguments, in this example it was "main"
# this uses sklearn.metrics.f1_score, you can pass in 
x.score(verbose=True)


Scoring Model
tags



Accuracy Score:  0.8096518987341772
Label Score:  [0.802380360695814, 0.8154323398775469]


[[0.802380360695814, 0.8154323398775469], 0.7768987341772152]